<a href="https://colab.research.google.com/github/Gops-8/Gesture-Recognition/blob/master/Neural_Nets_Project_Starter_Code_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import zipfile
# with zipfile.ZipFile(r'/content/drive/My Drive/Project_data.zip', 'r') as zip_ref:
#     zip_ref.extractall(r'/content/drive/My Drive/Project_data_f')


In [2]:
import numpy as np
import os
# from scipy.misc import imread, imresize
from imageio import imread
from skimage.transform import resize
import datetime
import os
import math

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open(r'/content/drive/My Drive/Project_data_f/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open(r'/content/drive/My Drive/Project_data_f/Project_data/val.csv').readlines())
batch_size = 50#experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =  list(range(0,30,2))    
 
                     #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = math.floor(len(t) / batch_size)  # calculate the number of batches
        for batch in range(num_batches):               # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),126,126,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5))    # batch_labels is the one hot representation of the output
            for folder in range(batch_size):           # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx):    #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image = resize(image, (126,126))
                    batch_data[folder,idx,:,:,0] = image[:,:,0] / 255.0#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] / 255.0#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] / 255.0#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # # write the code for the remaining data points which are left after full batches

        remaining_batch = t[(num_batches*batch_size):]
        batch_size_rem = len(remaining_batch)
        print( 'Source path = ', source_path, '; batch size =', batch_size_rem)
        batch_data = np.zeros((batch_size_rem,len(img_idx),126,126,3))
        batch_labels = np.zeros((batch_size_rem,5))
        for folder in range(batch_size_rem):
            imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) 
            for idx,item in enumerate(img_idx): 
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    image = resize(image, (126,126))
                    batch_data[folder,idx,:,:,0] = image[:,:,0] / 255.0
                    batch_data[folder,idx,:,:,1] = image[:,:,1] / 255.0
                    batch_data[folder,idx,:,:,2] = image[:,:,2] / 255.0

            batch_labels[folder, int(t[folder + (batch_size)].strip().split(';')[2])] = 1
            return   batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/My Drive/Project_data_f/Project_data/train'
val_path = '/content/drive/My Drive/Project_data_f/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


In [12]:
next(generator(train_path, train_doc, batch_size))

Source path =  /content/drive/My Drive/Project_data_f/Project_data/train ; batch size = 50


KeyboardInterrupt: ignored

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
#write your model here
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same',input_shape=(15,126,126,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 3)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
optimiser = 'adam'                    #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 126, 126, 32)  2624      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 126, 126, 32)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 126, 126, 32)  128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 124, 124, 32)  27680     
_________________________________________________________________
activation_2 (Activation)    (None, 13, 124, 124, 32)  0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 124, 124, 32)  128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 62, 41, 32)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.001)                 # write the REducelronplateau code here# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Streaming output truncated to the last 5000 lines.
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/va

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


Streaming output truncated to the last 5000 lines.
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/va

KeyboardInterrupt: ignored

Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My Drive/Project_data_f/Project_data/val ; batch size = 0
Source path =  /content/drive/My